In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
import string
from nltk.tokenize import word_tokenize
import ir_datasets
import pandas as pd
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [3]:
import lucene
from org.apache.lucene.search import IndexSearcher
from org.apache.lucene.index  import DirectoryReader
from org.apache.lucene.queryparser.classic import QueryParser
from org.apache.lucene.search.similarities import BM25Similarity
from org.apache.lucene.search.similarities import LMDirichletSimilarity
from org.apache.lucene.search.similarities import LMJelinekMercerSimilarity
from org.apache.lucene.analysis.standard import StandardAnalyzer
import os,sys
from java.nio.file import Paths

In [4]:
from org.apache.lucene.analysis.standard import StandardAnalyzer
from org.apache.lucene.index import IndexWriter , IndexWriterConfig
from org.apache.lucene.store import SimpleFSDirectory , FSDirectory
import org.apache.lucene.document as document 
from org.apache.lucene.document import Document, Field
from java.io import File
import json
from org.apache.lucene.analysis.en import EnglishAnalyzer

In [5]:
lucene.initVM()

In [6]:
def text_preproc(query):
    tokens=word_tokenize(query)
    #print(tokens)
    filtered_text=""
    for token in tokens:
        #print("Token",token)
        if re.search('[a-zA-Z]',token):
            if (('http' not in token) and ('@' not in token) and ('<.*?>' not in token) and ('*'not in token)):
                token=ps.stem(token)
                token=token.strip("/")
                token=token.replace("'"," ")
                filtered_text=filtered_text+" "+token
    return filtered_text

In [7]:
output=[]
Docid=[]
def search_BM_25(index_path,q,r,k,b):
    analyzer=EnglishAnalyzer()
    directory=FSDirectory.open(File(index_path).toPath())
    searcher=IndexSearcher(DirectoryReader.open(directory))
    #k=float(input("enter the value of k"))
    #b=float(input("enter the value of b"))
    searcher.setSimilarity(BM25Similarity(k,b))
    query=QueryParser("Content",analyzer).parse(q)
    scoreDocs=searcher.search(query,5).scoreDocs
    t=0
    for scoreDoc in scoreDocs:
        if(t<scoreDoc.score):
            t=scoreDoc.score
    #output=[]
    for scoreDoc in scoreDocs:
        doc=searcher.doc(scoreDoc.doc)
        
        if doc.get("Docid") not in Docid:
            importance=dict()
            Docid.append(doc.get("Docid"))
            importance["requestID"]=doc.get("RequestId")
            string=doc.get("Facttext")
            string=string.encode("ascii", "ignore")
            string=string.decode()
            importance["factText"]=string[0:200]
            importance["unixTimestamp"]=int(doc.get("Timestamp"))
            importance["importance"]=scoreDoc.score
            importance["sources"]=[doc.get("Docid")]
            importance["streamID"]=doc.get("Docid")
            importance["informationNeeds"]=[r]
            output.append(importance)
    
   
        
#         print(scoreDoc.score,t,scoreDoc.score/t)
#     print("Number of documnets matched",len(scoreDocs))

In [8]:
output=[]
def search_LM_DS(index_path,q,r,MU):
    analyzer=EnglishAnalyzer()
    directory=FSDirectory.open(File(index_path).toPath())
    searcher=IndexSearcher(DirectoryReader.open(directory))
    #MU=float(input("enter the value of MU"))
    searcher.setSimilarity(LMDirichletSimilarity(MU))
    query=QueryParser("Content",analyzer).parse(q)
    scoreDocs=searcher.search(query,5).scoreDocs
    for scoreDoc in scoreDocs:
        doc=searcher.doc(scoreDoc.doc)
        
        if doc.get("Docid") not in Docid:
            importance=dict()
            Docid.append(doc.get("Docid"))
            importance["requestID"]=doc.get("RequestId")
            string=doc.get("Facttext")
            string=string.encode("ascii", "ignore")
            string=string.decode()
            importance["factText"]=string[0:200]
            importance["unixTimestamp"]=int(doc.get("Timestamp"))
            importance["importance"]=scoreDoc.score
            importance["sources"]=[doc.get("Docid")]
            importance["streamID"]=doc.get("Docid")
            importance["informationNeeds"]=[r]
            output.append(importance)

In [9]:
output=[]
Docid=[]
def search_LM_JM(index_path,q,r,LAMBDA):
    analyzer=EnglishAnalyzer()
    directory=FSDirectory.open(File(index_path).toPath())
    searcher=IndexSearcher(DirectoryReader.open(directory))
    #k=float(input("enter the value of k"))
    #b=float(input("enter the value of b"))
    searcher.setSimilarity(LMJelinekMercerSimilarity(LAMBDA))
    query=QueryParser("Content",analyzer).parse(q)
    
    scoreDocs=searcher.search(query,5).scoreDocs
    t=0
    for scoreDoc in scoreDocs:
        if(t<scoreDoc.score):
            t=scoreDoc.score
    #output=[]
    for scoreDoc in scoreDocs:
        doc=searcher.doc(scoreDoc.doc)
        
        if doc.get("Docid") not in Docid:
            importance=dict()
            Docid.append(doc.get("Docid"))
            importance["requestID"]=doc.get("RequestId")
            string=doc.get("Content")
            string=string.encode("ascii", "ignore")
            string=string.decode()
            importance["factText"]=string[0:200]
            importance["unixTimestamp"]=int(doc.get("Timestamp"))
            importance["importance"]=scoreDoc.score
            importance["sources"]=[doc.get("Docid")]
            importance["streamID"]=doc.get("Docid")
            importance["informationNeeds"]=[r]
            output.append(importance)

In [10]:
# output1= pd.DataFrame(output)
# output1["importance"] = output1["importance"] / output1["importance"].max()
# with open("./test3.json", "a") as out_file:
#     for idx,row in output1\
#         .iterrows():
        
#         out_file.write("%s\n" % json.dumps(dict(row)))

In [11]:
import os
def retrieve(queryAsDataFrame,indexPath):
    Docid.clear()
    k=1.2
    b=0.75
    #LAMBDA=0.6
    j=0
    #MU=0.6
    while(j<len(queryAsDataFrame)):
        #print(queryAsDataFrame['text'][j])
        sent=text_preproc(queryAsDataFrame['text'][j])
        #print(sent)
        r=queryAsDataFrame['query_id'][j]
        #search_LM_DS(indexPath,sent,r,MU)
        search_BM_25(indexPath,sent,r,k,b)
        j+=1
    output1= pd.DataFrame(output)
    output.clear()
    output1["importance"] = output1["importance"] / output1["importance"].max()
    with open("./test11.json", "a") as out_file:
        for idx,row in output1\
             .iterrows():
        
            out_file.write("%s\n" % json.dumps(dict(row)))

    
#     json_object = json.dumps(output, indent=4)
#     with open("test.json", "a") as outfile:
#         outfile.write(json_object)
    

In [12]:
def query(event,indexPath):
    dataset = ir_datasets.load(event)
    queryAsDataFrame=pd.DataFrame(dataset.queries_iter())
    retrieve(queryAsDataFrame,indexPath)


In [13]:
eventNoList = [
          "001", # Lilac Wildfire 2017
          "002", # Cranston Wildfire 2018
          "003", # Holy Wildfire 2018
          "004", # Hurricane Florence 2018
          "005", # 2018 Maryland Flood
          "006", # Saddleridge Wildfire 2019
          "007", # Hurricane Laura 2020
          "008" # Hurricane Sally 2020
]


In [14]:
import requests

# Gets the list of days for a specified event number, e.g. '001'
def getDaysForEventNo(eventNo):

  # We will download a file containing the day list for an event
  url = "http://trecis.org/CrisisFACTs/CrisisFACTS-"+eventNo+".requests.json"

  # Download the list and parse as JSON
  dayList = requests.get(url).json()

  # Print each day
  # Note each day object contains the following fields
  #   {
  #      "eventID" : "CrisisFACTS-001",
  #      "requestID" : "CrisisFACTS-001-r3",
  #      "dateString" : "2017-12-07",
  #      "startUnixTimestamp" : 1512604800,
  #      "endUnixTimestamp" : 1512691199
  #   }

  return dayList

In [15]:
for eventNo in eventNoList: # for each event
    dayList = getDaysForEventNo(eventNo) # get the list of days
    print("Event "+eventNo)
    #print(dayList[0]['requestID'])
    for day in dayList: # for each day
        event="crisisfacts/"+eventNo+"/"+day["dateString"]
        path="crisisfacts_"+str(eventNo)+"_"+str(day["dateString"])
        parent_dir="/home/subinay/Trec 2022/CrisisFacts/index/"
        indexPath=os.path.join(parent_dir,path)
        query(event,indexPath)
        print("##")
        
    



Event 001
##
##
##
##
##
##
##
##
##
Event 002
##
##
##
##
##
##
Event 003
##
##
##
##
##
##
##
Event 004
##
##
##
##
##
##
##
##
##
##
##
##
##
##
##
Event 005
##
##
##
##
Event 006
##
##
##
##
Event 007
##
##
Event 008
##
##
##
##
##
##
##
##
